# 6章 学習に関するテクニック

## 誤差逆伝播法と数値微分の計算量の違い

> 確かに数値微分はパラメータ毎の勾配を求めるためにはネットワーク全体を2回順伝播することになるのに対して、誤差逆伝播法の場合は順伝播と逆伝播の合計 2 回だけで演算終わるもんなー。なので、数値微分自体の計算が重いというよりは、数値微分の場合はレイヤ数に比例して計算量が増えるので重いと言った方が適切なんだろうな。

数値微分は、確かに各パラメータの損失関数に対する微分を算出する際に、２回分損失関数の出力を必要とするため(f(x+h), f(x-h))  
単純に順伝播を２回していることになる  => 計算量的には、`パラメータ × 順伝播2回`

一方で誤差逆伝播法は、順伝播, 逆伝播の２回で終了する

すなわち、パラメータの個数分、すなわち層の個数分に比例して、計算量は大きくなると考えられる


## 重みの値

重みは均等になると、学習ができなくなる

In [3]:
# バッチ
# エポック・・・学習の単位
 
import numpy as np
from dataset.mnist import load_mnist
# 重みを0にした
from sample.two_layer_net_weight_zero import TwoLayerNet

(x_train, t_train), (x_test, t_test) = \
         load_mnist(normalize=True, one_hot_label=True)
    
train_loss_list = []
train_acc_list = []
test_acc_list = []

#  ハイパーパラメータ
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

# １エポックあたりの繰り返し数
iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # ミニバッチの取得
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配の計算
    grad = network.gradient(x_batch, t_batch)
    
    # パラメータの更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 学習経過の記録
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + " , " + str(test_acc))

train acc, test acc | 0.11236666666666667 , 0.1135
train acc, test acc | 0.13686666666666666 , 0.1399
train acc, test acc | 0.2903833333333333 , 0.2915
train acc, test acc | 0.3122 , 0.3124
train acc, test acc | 0.3458 , 0.3486
train acc, test acc | 0.34868333333333335 , 0.351
train acc, test acc | 0.34605 , 0.3404
train acc, test acc | 0.38356666666666667 , 0.3763
train acc, test acc | 0.49648333333333333 , 0.4916
train acc, test acc | 0.5725 , 0.5681
train acc, test acc | 0.66765 , 0.6606
train acc, test acc | 0.7544166666666666 , 0.754
train acc, test acc | 0.7919666666666667 , 0.7936
train acc, test acc | 0.8025833333333333 , 0.8086
train acc, test acc | 0.8167833333333333 , 0.8202
train acc, test acc | 0.8251 , 0.8288
train acc, test acc | 0.8292333333333334 , 0.8351


**重み０ではない時**

train acc, test acc | 0.0987166666667 , 0.098  
train acc, test acc | 0.787033333333 , 0.7893  
train acc, test acc | 0.878083333333 , 0.8841  
train acc, test acc | 0.900033333333 , 0.9034  
train acc, test acc | 0.909316666667 , 0.9142  
train acc, test acc | 0.915766666667 , 0.9188  
train acc, test acc | 0.921466666667 , 0.9227  
train acc, test acc | 0.924983333333 , 0.9265  
train acc, test acc | 0.928783333333 , 0.9313  
train acc, test acc | 0.9324 , 0.9328  
train acc, test acc | 0.93515 , 0.9361  
train acc, test acc | 0.937483333333 , 0.9387  
train acc, test acc | 0.939766666667 , 0.9405  
train acc, test acc | 0.942516666667 , 0.9425  
train acc, test acc | 0.944966666667 , 0.9435  
train acc, test acc | 0.946833333333 , 0.9451  
train acc, test acc | 0.948216666667 , 0.947  


## 6.1 パラメータの更新方法
* SGD : 確率的勾配降下法・・・・勾配を使って、勾配方向にパラメータを更新する
* Momentum・・・・重みパラメータに、速度などの概念を導入する
* AdaGrad・・・・パラメータの要素ごとに適応的に学習係数を調整しながら学習を行う手法
* Adam・・・・AdaGrad + Momenutmって感じ、難しいから詳細は論文参考

現在は, `SGD or Adam`って感じ

## 6.2 重みの初期値

*活性化関数の後の出力データの分布が適切に広がっていると、学習が早く正確に進む*

* 重みの初期値は、均一な値 or 小さすぎる値になってはいけない => **勾配損失**によって学習が進まなくなるから

活性化関数が線形である時
* Xavier の初期値 : $ \frac{1}{\sqrt{n}}$ の標準偏差を持つガウス分布で初期化

Reluの場合の時
* Heの初期値 : $ \sqrt{\frac{2}{n}}$の標準偏差を持つガウス分布で初期化

## 6.3 Batch Normalization

各レイヤの間に、アクティベーション(活性化関数の後の出力データ)の分布を適度に設定する処理を挟むこと  
具体的には、
1. データの分布が平均が 0で分散が 1 になるように正規化を行う  
2. この正規化されたデータに対して、固有のスケー ルとシフトで変換を行う (ここでのスケール、シフトの大きさは学習によって最適な値をセット)

**利点**
* 学習時間の短縮
* 初期値の依存性を減らす = 「初期値にロバスト」
* 過学習の抑制

## 6.4 正則化
**過学習の原因**
* パラメータを大量に持ち、表現力の高いモデルであること
* 訓練データが少ないこと

・・・訓練データとテストデータの乖離が大きいと時が一般に過学習と呼ばれる

**対策**
* Weight Decay
  * 重みを W とすれば、L2 ノルムの Weight decay は、$\frac{1}{2} \lambda W^{2}$ になり、これを損失関数に加算します。  
    ここで、λ は正則化の強さをコントロールするハイパーパラメーター
  * > 一般的にoverfitを防ぐための常套手段は、「何がoverfitっぽいか」ということを数値化してそれを損失関数に加えること 今回は、「一部の係数だけやたらデカい」=> 「Wがデカい」とoverfitっぽい、と考えているからL2ノルムを設定した。

* Dropout
  * ニューロンをランダムに消去しながら学習する手法
  
**アンサンブル学習**
> 機械学習では、アンサンブル学習というものをよく使います。アンサンブル学習とは、複数のモデルを個別に学習させ、推論時には、その複数の出力を平均するというものです。ニューラルネットワークの文脈で話をすると、たとえば、 5つの同じ構造(もしくは似た構造)のネットワークを用意して、それぞれに学習させ、テストのときには、その5つの出力の平均を答えとします。アンサ ンブル学習を行うことで、ニューラルネットワークの認識精度が数 % 向上する ことが実験的に分かっています。

## 6.5 ハイパーパラメータの検証

**ハイパーパラメータの種類**
* 各層のニューロンの数
* バッチサイズ
* パラメーター更新の学習係数
* Weight Decay


これらを決定するために、訓練データやテストデータとは別に`検証データ`というものを用意する

**ハイパーパラメーターの決定法**  
・・・・*基本的には、ランダムサンプリングによる探索*

1. ハイパーパラメータの範囲を設定する
2. 設定されたハイパーパラメータの範囲から、ランダムにサンプリングする
3. ステップ 1 でサンプリングされたハイパーパラメータの値を使用して学習を行い、検証データで認識精度を評価する(ただし、エポックは小さく設定)。
4. ステップ 1 とステップ 2 をある回数(100 回など)繰り返し、それらの認識精 度の結果から、ハイパーパラメータの範囲を狭める。

以上で、決定した範囲からハイパーパラメータを一つ選択して学習する  
また、ハイパーパラメー タの最適化において、より洗練された手法を求めるとすれば、`ベイズ最適化`などがある


